In [ ]:
## LSA 실습

import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import os

csv_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/abcnews-date-text.csv'

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

data = pd.read_csv(csv_filename, on_bad_lines='skip')
data

text = data[['headline_text']].copy()
text.head()
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

# 역토큰화(토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)
    
train_data = detokenized_doc

print(train_data)

train_data[:5]

# 상위 5000개의 단어만 사용
c_vectorizer = CountVectorizer(stop_words='english', max_features=5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

print('행렬의 크기 :',document_term_matrix.shape)

from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

print(lsa_model.components_.shape)

terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

## 토픽모델링
    - 문서의 집합에서 토픽을 찾아내는 프로세스

## LDA
    - LDA는 문서들이 토픽의 혼합으로 구성되어 있으며, 토픽들은 확률분포에 기반하여 단어들을 생성한다고 가정합니다. 그리고 데이터가 주어지면, LDA는 이 가정에 따라 단어들의 분포로부터 문서가 생성되는 과정을 역추적해 문서의 토픽을 찾아냄

# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

## scikit-learn LDA Model 활용
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

def print_top_words_as_probabilities(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic #{topic_idx + 1}:")
        
        # 확률로 변환하기 위해 토픽의 전체 비중 합으로 나눔
        topic_probabilities = topic / topic.sum()
        
        # 상위 단어 인덱스
        top_words_idx = topic_probabilities.argsort()[:-n_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        top_words_probs = topic_probabilities[top_words_idx]
        
        # 상위 단어와 그 확률 출력
        for word, prob in zip(top_words, top_words_probs):
            print(f"{word}: {prob:.4f}")

# 각 토픽에 대해 상위 10개의 단어 확률 출력
terms = tfidf_vectorizer.get_feature_names_out()
n_top_words = 10
print_top_words_as_probabilities(lda_model, terms, n_top_words)

## 형태소 분석기의 필요성
    - 한국어는 교착어(하나의 낱말이 하나의 어근과 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있는 언어), 이런 특성으로 인해 한국어는 영어와 달리 조사나 접사가 존재하며, 영어처럼 띄어쓰기 단위 토큰화가 제대로 동작하지 않는다.

en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

## 텍스트 분포를 이용한 비지도 학습 토크나이저(soynlp)
    - soynlp는 품사 태킹, 형태소분석등을 지원하는 한국어 형태소 분석기
    - 비지도 학습으로 형태소 분석을 한다는 특징을 갖고 있으며, 데이터에 자주 등장하는 단어들을 형태소 분석
    - soynlp 형태소 분석기는 내부적으로 단어 점수표로 동작합니다. 이 점수는 응집 확률(cohesion probability) 과 브랜칭 엔트로피(branching entropy) 를 활용

import urllib.request

txt_filename = os.getenv('HOME')+'/aiffel/topic_modelling/data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
corpus

i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

## soynlp는 학습과정을 거침
![image.png](attachment:image.png)
![image-2.png](attachment:image-2.png)

from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

word_score_table["반포한"].cohesion_forward

word_score_table["반포한강"].cohesion_forward

word_score_table["반포한강공"].cohesion_forward

word_score_table["반포한강공원"].cohesion_forward

word_score_table["반포한강공원에"].cohesion_forward

## 브랜칭 엔트로피
    - 확률 분포의 엔트로피값을 사용합니다. 이는 주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도

word_score_table["디스"].right_branching_entropy

word_score_table["디스플"].right_branching_entropy

word_score_table["디스플레"].right_branching_entropy

word_score_table["디스플레이"].right_branching_entropy

## soynlp의 LTokenizer
    - 한국어는 띄어쓰기 단위로 나눈 어절 토큰이 주로 L 토큰 + R 토큰의 형식을 가질 때가 많습니다. 예를 들어서 '공원에'는 '공원 + 에'로 나눌 수 있겠지요. 또는 '공부하는'은 '공부 + 하는'으로 나눌 수도 있을 것입니다. L 토크나이저는 L 토큰 + R 토큰으로 나누되, 점수가 가장 높은 L 토큰을 찾아내는 분리 기준

from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

## 최대 점수 토크나이저
     - 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저

from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")